In [1]:
import httpx
import concurrent.futures
from dataclasses import dataclass, asdict
import pandas as pd

In [2]:
@dataclass
class PlayerStats:
    name : int
    position : int
    positionInfo : int
    age : int
    dob : int
    nationality : str
    appearances : int
    goals : int
    headedGoals : int
    rightFootGoals : int
    leftFootGoals : int
    freeKickGoals : int
    penaltyWon : int
    penaltyScored : int
    penaltyMissed : int
    shots : int
    shotsOnTarget : int
    hitWoodwork : int
    bigChancesMissed : int

    #TeamPlay
    bigChancesCreated : int
    assist : int
    wins : int
    losses : int
    draws : int

    #Discpline
    yellowCards : int
    redCards : int
    fouls : int
    offsides : int

    # GK
    clean_sheets : int
    penaltyFaced : int
    penaltySaved : int
    penaltyConceded : int
    goalsConceded : int
    saves : int

    #DF
    tackles : int
    blockedShots : int
    interception : int
    clearances : int


In [3]:
def fetch_players(url, headers, querystring, player_ids):
    page = 0
    while True:
        querystring["page"] = str(page)
        response = httpx.get(url, headers=headers, params=querystring, timeout=10)
        json_data = response.json()
        if response.status_code == 200:
            content = json_data.get('content', [])
            if len(content) == 0:
                break
            for player_data in content:
                player_id = int(player_data.get('id', -1))
                player_ids.append(player_id)
            page += 1
        else:
            break

def player_API():
    seasons_dict = {
    "2023/24": 578,
    "2022/23": 489,
    "2021/22": 418,
    "2020/21": 363,
    "2019/20": 274,
    "2018/19": 210,
    "2017/18": 79,
    "2016/17": 54,
    "2015/16": 42,
    "2014/15": 27,
    "2013/14": 22,
    "2012/13": 21,
    "2011/12": 20,
    "2010/11": 19,
    "2009/10": 18,
    "2008/09": 17,
    "2007/08": 16,
    "2006/07": 15,
    "2005/06": 14,
    "2004/05": 13,
    "2003/04": 12,
    "2002/03": 11,
    "2001/02": 10,
    "2000/01": 9,
    "1999/00": 8,
    "1998/99": 7,
    "1997/98": 6,
    "1996/97": 5,
    "1995/96": 4,
    "1994/95": 3,
    "1993/94": 2,
    "1992/93": 1
}
    url = "https://footballapi.pulselive.com/football/players"
    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "^if-none-match": "W/^\^0b215d09edebc804ddb9ce7efb073e8c8^^^",
        "origin": "https://www.premierleague.com",
        "referer": "https://www.premierleague.com/",
        "^sec-ch-ua": "^\^Microsoft",
        "sec-ch-ua-mobile": "?0",
        "^sec-ch-ua-platform": "^\^Windows^^^",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
    }
    querystring = {"pageSize":"10","compSeasons":"","altIds":"true","page":"","type":"player","id":"-1","compSeasonId":""}
    player_ids = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for year, id in seasons_dict.items():
            querystring["compSeasonId"] = str(id)
            querystring["compSeasons"] = str(id)
            future = executor.submit(fetch_players, url, headers, querystring.copy(), player_ids)
            futures.append(future)

        # Wait for all futures to complete
        for future in concurrent.futures.as_completed(futures):
            pass

    return player_ids

In [4]:
# def get_playerStats(id_list):
#     player_data = []
#     querystring = {"comps":"1"}
#     headers = {
#     "accept": "*/*",
#     "accept-language": "en-US,en;q=0.9",
#     "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
#     "^if-none-match": "W/^\^02d0b206ded7f872f238c8cea7d26fa01^^^",
#     "origin": "https://www.premierleague.com",
#     "referer": "https://www.premierleague.com/",
#     "^sec-ch-ua": "^\^Microsoft",
#     "sec-ch-ua-mobile": "?0",
#     "^sec-ch-ua-platform": "^\^Windows^^^",
#     "sec-fetch-dest": "empty",
#     "sec-fetch-mode": "cors",
#     "sec-fetch-site": "cross-site",
#     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
# }

#     for id in id_list:
#         url = f"https://footballapi.pulselive.com/football/stats/player/{id}"
#         try:
#             response = httpx.get(url, headers=headers, params=querystring, timeout= 15)
#             stats_json = response.json()
#             if stats_json:
#                 player_data.append(stats_json)
#         except httpx.TimeoutException:
#             print("Request timed out.")
#     return player_data

def get_playerStats(id_list):
    player_data = []
    querystring = {"comps": "1"}
    headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "^if-none-match": "W/^\^02d0b206ded7f872f238c8cea7d26fa01^^^",
    "origin": "https://www.premierleague.com",
    "referer": "https://www.premierleague.com/",
    "^sec-ch-ua": "^\^Microsoft",
    "sec-ch-ua-mobile": "?0",
    "^sec-ch-ua-platform": "^\^Windows^^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
}

    def fetch_player_stats(player_id):
        url = f"https://footballapi.pulselive.com/football/stats/player/{player_id}"
        try:
            response = httpx.get(url, headers=headers, params=querystring, timeout=15)
            stats_json = response.json()
            if stats_json:
                return stats_json
        except httpx.TimeoutException:
            print(f"Request timed out for player ID {player_id}.")
        return None

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit each player ID to the thread pool for concurrent execution
        future_to_player = {executor.submit(fetch_player_stats, player_id): player_id for player_id in id_list}
        for future in concurrent.futures.as_completed(future_to_player):
            player_id = future_to_player[future]
            try:
                stats_json = future.result()
                if stats_json:
                    player_data.append(stats_json)
            except Exception as e:
                print(f"Exception occurred for player ID {player_id}: {e}")

    return player_data


In [5]:
def parse_playerStats(json):
    players_info = []
    for player in json:
        try:
            new_player = PlayerStats(
                name = player['entity']['name']['display'],
                position = player['entity']['info']['position'],
                positionInfo = player['entity']['info']['positionInfo'],
                age = player['entity']['age'],
                dob = player['entity']['birth']['date']['label'],
                nationality = player['entity']['nationalTeam']['country'],
                appearances = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "appearances"), 0)),

                #Attack
                goals  = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "goals"), 0)),
                headedGoals = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_hd_goal"), 0)),
                rightFootGoals = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_rf_goal"), 0)),
                leftFootGoals = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_lf_goal"), 0)),
                freeKickGoals = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_freekick_goal"), 0)),
                penaltyWon = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "penalty_won"), 0)),
                penaltyScored = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_pen_goal"), 0)),
                penaltyMissed = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "att_pen_miss"), 0)),
                shots = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "total_scoring_att"), 0)),
                shotsOnTarget = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "ontarget_scoring_att"), 0)),
                hitWoodwork = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "hit_woodwork"), 0)),
                bigChancesMissed = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "big_chance_missed"), 0)),
              
                # #TeamPlay
                bigChancesCreated = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "big_chance_created"), 0)),
                assist = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "goal_assist"), 0)),
                wins = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "wins"), 0)),
                losses = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "losses"), 0)),
                draws = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "draws"), 0)),

                #Discpline
                yellowCards = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "yellow_card"), 0)),
                redCards = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "red_card"), 0)),
                fouls = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "fouls"), 0)),
                offsides = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "offsides"), 0)),

                #GoalKeeper
                clean_sheets = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "clean_sheet"), 0)),
                penaltyFaced = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "penalty_faced"), 0)),
                penaltySaved = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "penalty_save"), 0)),
                penaltyConceded = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "penalty_conceded"), 0)),
                goalsConceded = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "goals_conceded"), 0)),
                saves = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "saves"), 0)),
                #Defence
                tackles = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "total_tackle"), 0)),
                blockedShots = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "blocked_scoring_att"), 0)),
                interception = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "interception"), 0)),
                clearances = int(next((stat["value"] for stat in player["stats"] if stat["name"] == "total_clearance"), 0)),
            )
            players_info.append(asdict(new_player))
        except Exception as e:
            print(f"Skipping entry due to missing key: {e}")
    return players_info


In [6]:
def createExcel(data):
    df = pd.DataFrame(data)
    # Save DataFrame to CSV
    df.to_excel('players_data.xlsx', index=False)

In [7]:
def main():
    player_ids = player_API()
    json = get_playerStats(player_ids)
    player_stats = parse_playerStats(json)
    createExcel(player_stats)

if __name__ == "__main__":
    main()

Exception occurred for player ID 1712: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 563: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 2491: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 2554: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 2869: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 168: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 1768: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 3256: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 1768: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 2554: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 2869: Expecting value: line 1 column 1 (char 0)
Exception occurred for player ID 1768: Expecting value: line 1 column 1 (char 0)
Exception occurred for player 